# Benchmarks

In this notebook we compare the performance of the two genetic algorithms against each other and against a random search and a dynamic programming algorithm.

## Disclaimer

This is a very simple benchmark and should not be taken as a definitive proof of the performance of the algorithms.
A more proper benchmark would require a more extensive set of tests and a more detailed analysis of the results.

It should also be noted that the genetic algorithms are stochastic and the results may vary from run to run.
As such multiple runs should be performed to have a more accurate estimate of the performance of the algorithms.

Moreover, other factors other than time should be considered to assess the quality of an algorithmm.

#### On genetic algorithms implementation

The goal of the project was to implement genetic algorithms for the graph coloring problem.
We implemented a problem-specific version of the algorithm that requires fewer generations to converge.
Pure **time** performance was not the main goal of the project.
As such the implementation could be improved to be faster.


In [28]:
import sys
sys.path.append('..')
import ga4graphcoloring as ga
import numpy as np
import time

In [29]:
def score(solution, graph):
    fitness = 0
    # color of a vertex is given by the value stored the corresponding index of the individual
    for color, adj_row in zip(solution, graph.adj_matrix):
        # check if the vertex has the same color as any of its neighbours
        for vertex in adj_row.nonzero()[0]:  # only iter on adjacent vertices
            if color == solution[vertex]:
                fitness += 1
    # fitness is halved as we count each edge twice
    return fitness // 2

# define random search algorithm
def random_search(max_colors: int, graph: ga.Graph, max_iter=100_000):
    best_fitness = np.inf
    best_solution = None
    for i in range(max_iter):
        solution = np.random.randint(0, max_colors, graph.n_vertices)
        fitness = score(solution, graph)
        if fitness < best_fitness:
            best_score = score
            best_solution = solution
    return best_solution, best_fitness

# define dynamic programming algorithm
def dynamic_programming(max_colors: int, graph: ga.Graph):
    """Original [source](https://www.geeksforgeeks.org/m-coloring-problem/) has been adapted to our graph representation."""
    
    def is_safe(vertex, color, color_assignment):
        for v in range(len(graph.adj_matrix)):
            if graph.adj_matrix[vertex][v] and v in color_assignment and color_assignment[v] == color:
                return False
        return True

    def dfs(vertex, color_assignment):
        if vertex not in color_assignment:
            for color in range(max_colors):
                if is_safe(vertex, color, color_assignment):
                    color_assignment[vertex] = color
                    if all(dfs(next_vertex, color_assignment) for next_vertex in range(len(graph.adj_matrix))):
                        return True
                    color_assignment.pop(vertex)
            return False
        return True

    color_assignment = {}
    if dfs(0, color_assignment):
        return True, color_assignment
    else:
        return False, {}


In [31]:
# generate a random graph
N_VERTICES = 200
DENSITY = 0.54

graph = ga.Graph(N_VERTICES, DENSITY)

In [32]:
def benchmark(max_colors: int, graph: ga.Graph, max_evolutions=250):
    times = []
    founds = []

    # dynamic programming
    print('Dynamic programming')
    start = time.time()
    found, _ = dynamic_programming(max_colors, graph)
    end = time.time()

    times.append(end - start)
    founds.append(found)
    print(f'Best fitness: {0}, time: {end - start:.2f}, found: {found}')
    
    # random search
    print('Random search')
    start = time.time()
    best_solution, best_fitness = random_search(max_colors, graph)
    end = time.time()
    
    times.append(end - start)
    founds.append(best_fitness == 0)
    print(f'Best fitness: {best_fitness}, time: {end - start:.2f}, found: {best_fitness == 0}')    
    
    # standard ga
    print('Standard GA')
    pop = ga.Population(max_colors, 100, graph)
    
    start = time.time()
    for i in range(max_evolutions):
        pop.evolve()
        if pop.best_fitness == 0:
            break
    end = time.time()
    
    times.append(end - start)
    founds.append(pop.best_fitness == 0)
    print(f'Best fitness: {pop.best_fitness}, time: {end - start:.2f}, found: {pop.best_fitness == 0}')
    
    # smart ga
    print('Smart GA')
    pop = ga.SmartPopulation(max_colors, 50, graph)
    
    start = time.time()
    for i in range(max_evolutions):
        pop.evolve()
        if pop.best_fitness == 0:
            break
    end = time.time()
    
    times.append(end - start)
    founds.append(pop.best_fitness == 0)
    print(f'Best fitness: {pop.best_fitness}, time: {end - start:.2f}, found: {pop.best_fitness == 0}')
    return times, founds

In [33]:
benchmark(62, graph)

Dynamic programming
Best fitness: 0, time: 0.06, found: True
Random search
Best fitness: inf, time: 415.25, found: False
Standard GA
Best fitness: 1, time: 1039.00, found: False
Smart GA
Best fitness: 0, time: 3.50, found: True


([0.06263422966003418,
  415.2542190551758,
  1039.003441810608,
  3.4980647563934326],
 [True, False, False, True])

## Conclusions

The _dynamic programming_ algorithm is the fastest.
Both _dynamic programming_ and _smart GA_ algorithms are able to find the optimal solution in a reasonable amount of time.
As expected, the _random search_ algorithm is  not able to find the optimal solution in a reasonable amount of time.

The _standard GA_ algorithm is not able to find the optimal solution in a reasonable amount of generations.
This is probably due to the fact that the algorithm gets stuck in a local optima. To reach the global optima many vertexes need to change color at the same time, which is unlikely to happen in a single generation.
